In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

train_images = pd.read_pickle('D:/Study/Fall2019/COMP551/551A3/train_max_x.zip') 
x_train = np.reshape(train_images, (50000, 128, 128))
x_train = x_train.reshape(x_train.shape[0], 128, 128, 1)

df = pd.read_csv('D:/Study/Fall2019/COMP551/551A3/train_y_mod.csv')
y_train = df.Label.to_numpy()

#X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


# Reshaping the array to 4-dims so that it can work with the Keras API
#X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
#X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)
input_shape = (128, 128, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
#X_test = X_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255.0
#X_test /= 255.0
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
#print('Number of images in x_test', X_test.shape[0])

x_train shape: (50000, 128, 128, 1)
Number of images in x_train 50000


In [2]:
#x_train = x_train.astype('float32')
#x_train /= 255.0
x_train[x_train < 0.8] = 0
x_train[x_train >= 0.8] = 1


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.keras import backend
from tensorflow.python.keras import initializers
from tensorflow.python.keras import layers
from tensorflow.python.keras import models
from tensorflow.python.keras import regularizers
#from official.vision.image_classification import imagenet_preprocessing

L2_WEIGHT_DECAY = 1e-4
BATCH_NORM_DECAY = 0.9
BATCH_NORM_EPSILON = 1e-5


def _gen_l2_regularizer(use_l2_regularizer=True):
  return regularizers.l2(L2_WEIGHT_DECAY) if use_l2_regularizer else None


def identity_block(input_tensor,
                   kernel_size,
                   filters,
                   stage,
                   block,
                   use_l2_regularizer=True):
  """The identity block is the block that has no conv layer at shortcut.
  Args:
    input_tensor: input tensor
    kernel_size: default 3, the kernel size of middle conv layer at main path
    filters: list of integers, the filters of 3 conv layer at main path
    stage: integer, current stage label, used for generating layer names
    block: 'a','b'..., current block label, used for generating layer names
    use_l2_regularizer: whether to use L2 regularizer on Conv layer.
  Returns:
    Output tensor for the block.
  """
  filters1, filters2, filters3 = filters
  if backend.image_data_format() == 'channels_last':
    bn_axis = 3
  else:
    bn_axis = 1
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  x = layers.Conv2D(
      filters1, (1, 1),
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name=conv_name_base + '2a')(
          input_tensor)
  x = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name=bn_name_base + '2a')(
          x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(
      filters2,
      kernel_size,
      padding='same',
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name=conv_name_base + '2b')(
          x)
  x = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name=bn_name_base + '2b')(
          x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(
      filters3, (1, 1),
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name=conv_name_base + '2c')(
          x)
  x = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name=bn_name_base + '2c')(
          x)

  x = layers.add([x, input_tensor])
  x = layers.Activation('relu')(x)
  return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2),
               use_l2_regularizer=True):
  """A block that has a conv layer at shortcut.
  Note that from stage 3,
  the second conv layer at main path is with strides=(2, 2)
  And the shortcut should have strides=(2, 2) as well
  Args:
    input_tensor: input tensor
    kernel_size: default 3, the kernel size of middle conv layer at main path
    filters: list of integers, the filters of 3 conv layer at main path
    stage: integer, current stage label, used for generating layer names
    block: 'a','b'..., current block label, used for generating layer names
    strides: Strides for the second conv layer in the block.
    use_l2_regularizer: whether to use L2 regularizer on Conv layer.
  Returns:
    Output tensor for the block.
  """
  filters1, filters2, filters3 = filters
  if backend.image_data_format() == 'channels_last':
    bn_axis = 3
  else:
    bn_axis = 1
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  x = layers.Conv2D(
      filters1, (1, 1),
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name=conv_name_base + '2a')(
          input_tensor)
  x = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name=bn_name_base + '2a')(
          x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(
      filters2,
      kernel_size,
      strides=strides,
      padding='same',
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name=conv_name_base + '2b')(
          x)
  x = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name=bn_name_base + '2b')(
          x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(
      filters3, (1, 1),
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name=conv_name_base + '2c')(
          x)
  x = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name=bn_name_base + '2c')(
          x)

  shortcut = layers.Conv2D(
      filters3, (1, 1),
      strides=strides,
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name=conv_name_base + '1')(
          input_tensor)
  shortcut = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name=bn_name_base + '1')(
          shortcut)

  x = layers.add([x, shortcut])
  x = layers.Activation('relu')(x)
  return x


def resnet50(num_classes,
             batch_size=None,
             use_l2_regularizer=True,
             rescale_inputs=False):
  """Instantiates the ResNet50 architecture.
  Args:
    num_classes: `int` number of classes for image classification.
    batch_size: Size of the batches for each step.
    use_l2_regularizer: whether to use L2 regularizer on Conv/Dense layer.
    rescale_inputs: whether to rescale inputs from 0 to 1.
  Returns:
      A Keras model instance.
  """
  input_shape = (128, 128, 1)
  img_input = layers.Input(shape=input_shape, batch_size=batch_size)
  if rescale_inputs:
    # Hub image modules expect inputs in the range [0, 1]. This rescales these
    # inputs to the range expected by the trained model.
    x = layers.Lambda(
        lambda x: x * 255.0 - backend.constant(
            imagenet_preprocessing.CHANNEL_MEANS,
            shape=[1, 1, 3],
            dtype=x.dtype),
        name='rescale')(
            img_input)
  else:
    x = img_input

  if backend.image_data_format() == 'channels_first':
    x = layers.Lambda(
        lambda x: backend.permute_dimensions(x, (0, 3, 1, 2)),
        name='transpose')(x)
    bn_axis = 1
  else:  # channels_last
    bn_axis = 3

  x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(x)
  x = layers.Conv2D(
      64, (7, 7),
      strides=(2, 2),
      padding='valid',
      use_bias=False,
      kernel_initializer='he_normal',
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name='conv1')(
          x)
  x = layers.BatchNormalization(
      axis=bn_axis,
      momentum=BATCH_NORM_DECAY,
      epsilon=BATCH_NORM_EPSILON,
      name='bn_conv1')(
          x)
  x = layers.Activation('relu')(x)
  x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

  x = conv_block(
      x,
      3, [64, 64, 256],
      stage=2,
      block='a',
      strides=(1, 1),
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [64, 64, 256],
      stage=2,
      block='b',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [64, 64, 256],
      stage=2,
      block='c',
      use_l2_regularizer=use_l2_regularizer)

  x = conv_block(
      x,
      3, [128, 128, 512],
      stage=3,
      block='a',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [128, 128, 512],
      stage=3,
      block='b',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [128, 128, 512],
      stage=3,
      block='c',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [128, 128, 512],
      stage=3,
      block='d',
      use_l2_regularizer=use_l2_regularizer)

  x = conv_block(
      x,
      3, [256, 256, 1024],
      stage=4,
      block='a',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [256, 256, 1024],
      stage=4,
      block='b',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [256, 256, 1024],
      stage=4,
      block='c',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [256, 256, 1024],
      stage=4,
      block='d',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [256, 256, 1024],
      stage=4,
      block='e',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [256, 256, 1024],
      stage=4,
      block='f',
      use_l2_regularizer=use_l2_regularizer)

  x = conv_block(
      x,
      3, [512, 512, 2048],
      stage=5,
      block='a',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [512, 512, 2048],
      stage=5,
      block='b',
      use_l2_regularizer=use_l2_regularizer)
  x = identity_block(
      x,
      3, [512, 512, 2048],
      stage=5,
      block='c',
      use_l2_regularizer=use_l2_regularizer)

  rm_axes = [1, 2] if backend.image_data_format() == 'channels_last' else [2, 3]
  x = layers.Lambda(lambda x: backend.mean(x, rm_axes), name='reduce_mean')(x)
  x = layers.Dense(
      num_classes,
      kernel_initializer=initializers.RandomNormal(stddev=0.01),
      kernel_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      bias_regularizer=_gen_l2_regularizer(use_l2_regularizer),
      name='fc1000')(
          x)

  # A softmax that is followed by the model loss must be done cannot be done
  # in float16 due to numeric issues. So we pass dtype=float32.
  x = layers.Activation('softmax', dtype='float32')(x)

  # Create model.
  return models.Model(img_input, x, name='resnet50')

In [4]:
model = resnet50(10,
             batch_size=32,
             use_l2_regularizer=True,
             rescale_inputs=False)


model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
for i in range(15):
    print("epoch", i)
    model.fit(x=x_train[0:49984] ,y=y_train[0:49984], epochs=1, batch_size = 32)
    model.fit(x=x_train[16:50000] ,y=y_train[16:50000], epochs=1, batch_size = 32)

epoch 0
49984/49984 [==============================] - 183s 4ms/sample - loss: 1.1058 - acc: 0.8482
epoch 1
49984/49984 [==============================] - 184s 4ms/sample - loss: 0.7357 - acc: 0.9071
epoch 2
26976/49984 [===============>..............] - ETA: 1:24 - loss: 0.6197 - acc: 0.9252

In [13]:
model.fit(x=x_train[0:49984] ,y=y_train[0:49984], epochs=5, batch_size = 32)

Epoch 1/5
49984/49984 [==============================] - 181s 4ms/sample - loss: 0.2497 - acc: 0.9713
Epoch 2/5
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2531 - acc: 0.9711
Epoch 3/5
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2462 - acc: 0.9712
Epoch 4/5
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2423 - acc: 0.9719
Epoch 5/5
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2399 - acc: 0.9723


In [14]:
model.fit(x=x_train[0:49984] ,y=y_train[0:49984], epochs=5, batch_size = 32)

Epoch 1/5
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2349 - acc: 0.9725
Epoch 2/5
49984/49984 [==============================] - 183s 4ms/sample - loss: 0.2255 - acc: 0.9760
Epoch 3/5
49984/49984 [==============================] - 183s 4ms/sample - loss: 0.2280 - acc: 0.9738
Epoch 4/5
49984/49984 [==============================] - 183s 4ms/sample - loss: 0.2325 - acc: 0.9722
Epoch 5/5
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2262 - acc: 0.9747


In [15]:
model.fit(x=x_train[0:49984] ,y=y_train[0:49984], epochs=1, batch_size = 32)

49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2226 - acc: 0.9753


In [21]:
result_images = pd.read_pickle('D:/Study/Fall2019/COMP551/551A3/test_max_x.zip') 
x_test = np.reshape(result_images, (10000, 128, 128))
x_test = np.reshape(x_test, (10000, 128, 128, 1))

x_test = x_test.astype('float32')
x_test /= 255.0

x_test[x_test < 0.8] = 0
x_test[x_test >= 0.8] = 1

In [22]:
y_result = model.predict(x=x_test)
result = []
for i in range(10000):
    result.append(np.argmax(y_result[i]))

In [23]:
import csv
with open('D:/Study/Fall2019/COMP551/551A3/submitC.csv', mode='w') as writingFile:
    writer = csv.writer(writingFile)
    writer.writerow(['id', 'Label'])
    for index in range(10000):
        writer.writerow([index, result[index]])

In [19]:
model.fit(x=x_train[0:49984] ,y=y_train[0:49984], epochs=15, batch_size = 32)

Epoch 1/15
49984/49984 [==============================] - 180s 4ms/sample - loss: 0.2217 - acc: 0.9745
Epoch 2/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2193 - acc: 0.9757
Epoch 3/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2199 - acc: 0.9752
Epoch 4/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2158 - acc: 0.9758
Epoch 5/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2185 - acc: 0.9754
Epoch 6/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2182 - acc: 0.9750
Epoch 7/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2109 - acc: 0.9770
Epoch 8/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2092 - acc: 0.9767
Epoch 9/15
49984/49984 [==============================] - 183s 4ms/sample - loss: 0.2117 - acc: 0.9759
Epoch 10/15
49984/49984 [==============================] - 182s 4ms/sampl

In [20]:
model.fit(x=x_train[0:49984] ,y=y_train[0:49984], epochs=15, batch_size = 32)

Epoch 1/15
49984/49984 [==============================] - 181s 4ms/sample - loss: 0.1981 - acc: 0.9788
Epoch 2/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2004 - acc: 0.9777
Epoch 3/15
49984/49984 [==============================] - 183s 4ms/sample - loss: 0.1998 - acc: 0.9780
Epoch 4/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2008 - acc: 0.9776
Epoch 5/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.1980 - acc: 0.9778
Epoch 6/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.1900 - acc: 0.9802
Epoch 7/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.2027 - acc: 0.9770
Epoch 8/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.1889 - acc: 0.9803
Epoch 9/15
49984/49984 [==============================] - 182s 4ms/sample - loss: 0.1942 - acc: 0.9770
Epoch 10/15
49984/49984 [==============================] - 182s 4ms/sampl

In [ ]:
model.fit(x=x_train[0:49984] ,y=y_train[0:49984], epochs=5, batch_size = 32)